In [ ]:
import sys
sys.path.append('../script/')
from os.path import exists
import gc
from functools import partial

import numpy as np
import pandas as pd
import optuna

import utils
import models
import train as trainer

DEVICE = "cuda"
EPOCHS = 2

## Add folds No. for CV

In [ ]:
path_fold = "../input/folds/train_folds.csv"
if not exists(path_fold):
    from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
    df = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
    df.loc[:, "kfold"] = -1
    df = df.sample(frac=1).reset_index(drop=True)
    targets = df.drop("sig_id", axis=1).values

    mskf = MultilabelStratifiedKFold(n_splits=5)
    for fold_, (tr_, va_) in enumerate(mskf.split(X=df, y=targets)):
        df.loc[va_, "kfold"] = fold_
    df.to_csv(path_fold, index=False)

## Params for training function `run_training`

In [ ]:
fold = 0
# optuna 使わないので適当
params = {
    "num_layers": 3,
    "hidden_size": 16,
    "dropout": 0.3,
    "learning_rate": 1e-3,
}
save_model=True

---
## Prototyping training process from HERE

In [ ]:
df = pd.read_csv("../input/folds/train.csv")
with open("../input/folds/targets", "r") as f:
    targets = f.read().split("\n")
with open("../input/folds/features", "r") as f:
    features = f.read().split("\n")

In [5]:
print(f'[Fold No.{fold:>3}]\n')
# train_df = df[df.kfold != fold].reset_index(drop=True)
train_df = df[df.kfold != 1].reset_index(drop=True)
valid_df = df[df.kfold == fold].reset_index(drop=True)

[Fold No.  0]



In [6]:
x_tr = train_df[features].astype(np.float32).to_numpy()
x_va = valid_df[features].astype(np.float32).to_numpy()

In [7]:
y_tr = train_df[targets].astype(np.int8).to_numpy()
y_va = valid_df[targets].astype(np.int8).to_numpy()
# Trim aux target
y_tr = y_tr[:,:-1].copy()
y_va = y_va[:,:-1].copy()

In [8]:
model = "rf"
if model == "svm":
    from sklearn.svm import SVC
    from sklearn.multiclass import OneVsRestClassifier
    model = OneVsRestClassifier(SVC(probability=True, verbose=True))
if model == "rf":
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier(n_jobs=4, verbose=True)

In [ ]:
%%time
model.fit(x_tr, y_tr)

In [ ]:
pred = model.predict(x_va)